In [3]:
# Concise Implementation of Linear Regression
# Step 1. Generating the Dataset
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

# Possible since it is the simulatioin study for clarity of understanding the workflow.
# Not possible in the real-world data analysis
true_w = tf.constant([2, -3.4]) # true weights
true_b = 4.2 # true bias
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # design matrix and the response vector (supervised learning)

# Step 2. Reading the Dataset
# Splitting the training data set into minibatches for the purpose of training, where minibatch SGD is used for iterative process of function estimation.
def load_array(data_arrays, batch_size, is_train=True): #@save
    """"Construct a Tensorflow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays) # 그냥 dataset 만드는 함수로 이해하면 편할 것 같다.
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10 # size of respective minibatches
data_iter = load_array((features, labels), batch_size) # BatchDataset of size 100 (=1000/10)
#next(iter(data_iter))

# Step 3. Defining the Model
# 'keras' is the high-level API for Tensorflow
# In this case, fully-connected multilayer perceptron of single layer is assumed!
#net = tf.keras.Sequential()
#net.add(tf.keras.layers.Dense(1))

# Step 4. Initializing Model Parameters
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

# Step 5. Defining the Loss Function
loss = tf.keras.losses.MeanSquaredError()

# Step 6. Defining the Optimization Algorithm
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

# Step 7. Training
num_epochs = 3 # can be understood as an hyperparameter
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y) # calculates the loss function value using a given minibatch
        grads = tape.gradient(l, net.trainable_variables) # gradient w.r.t weights and bias, respectively -> reason why the ftn zip() is used in the following line!
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels) # for final evaluation of loss function at the end of each epoch
    print(f'epoch {epoch + 1}, loss {l:f}')

# Step 8. Additional Step of Checking Goodness-of-fit (possible since it is a synthetic data with known true parameters)
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - true_b - b)

epoch 1, loss 0.000348
epoch 2, loss 0.000105
epoch 3, loss 0.000105
error in estimating w tf.Tensor([-0.00083876 -0.00065589], shape=(2,), dtype=float32)
error in estimating b [-4.20046]
